# 02 — Pathlib e I/O (CSV, JSON, Parquet)

Objetivo: dominar leitura/escrita de arquivos e organização de pastas para projetos de dados.

Tempo: ~25 min

In [ ]:
from pathlib import Path

def find_repo_root(start: Path | None = None) -> Path:
    """Sobe diretórios até encontrar uma 'marca' do repositório (README.md + pasta data)."""
    cur = (start or Path.cwd()).resolve()
    for _ in range(10):
        if (cur / "README.md").exists() and (cur / "data").exists():
            return cur
        cur = cur.parent
    # fallback: assume cwd
    return Path.cwd().resolve()

ROOT = find_repo_root()
DATA_DIR = ROOT / "data"
SAMPLE_DIR = DATA_DIR / "sample"

print("ROOT:", ROOT)
print("SAMPLE_DIR:", SAMPLE_DIR)

## 1) Pathlib na prática

`Path` deixa seu código portável (Windows/Mac/Linux) e mais legível.

In [ ]:
from pathlib import Path

raw = ROOT / "data" / "raw"
processed = ROOT / "data" / "processed"
output = ROOT / "data" / "output"

# Criar pastas de trabalho (as .gitkeep já existem, mas este padrão é útil em projetos reais)
for p in (raw, processed, output):
    p.mkdir(parents=True, exist_ok=True)

print("raw:", raw)
print("processed:", processed)
print("output:", output)

## 2) Leitura/Escrita de CSV (pandas)

Você vai fazer isso 500x por ano. Padrão recomendado: sempre defina `dtype` quando possível e trate datas.

In [ ]:
import pandas as pd

sales = pd.read_csv(SAMPLE_DIR / "sales.csv")
sales["date"] = pd.to_datetime(sales["date"])

sales.to_csv(output / "sales_export.csv", index=False)

sales.head()

## 3) JSON (bom para logs e snapshots)

Aqui vamos salvar um 'resumo' do dataset.

In [ ]:
import json

summary = {
    "rows": int(len(sales)),
    "columns": list(sales.columns),
    "date_min": str(sales["date"].min().date()),
    "date_max": str(sales["date"].max().date()),
    "revenue_total": float(sales["revenue"].sum()),
}

(out := output / "sales_summary.json").write_text(json.dumps(summary, ensure_ascii=False, indent=2), encoding="utf-8")
print("salvo em:", out)

summary

## 4) Parquet (padrão forte em dados)

Parquet é eficiente para leitura/escrita e compactação. Aqui usamos `pyarrow` por baixo.

In [ ]:
# Escrevendo parquet
parquet_path = output / "sales.parquet"
sales.to_parquet(parquet_path, index=False)

# Lendo parquet
sales_pq = pd.read_parquet(parquet_path)
sales_pq.head()

## Exercícios

1- Salve um CSV apenas com colunas `date`, `region`, `category`, `revenue`.
2- Crie um JSON com receita total por `region`.
3- Compare tamanho do CSV vs Parquet no seu disco (use `.stat().st_size`).

In [ ]:
# Escreva suas respostas aqui
